In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Define the paths
base_image_dir = '../archive/gaussian_filtered_images'
labels_path = '../archive/train.csv'

# Load the labels
labels_df = pd.read_csv(labels_path)

# Map the labels to the corresponding directories
label_map = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR'
}

In [3]:
# Create image paths and labels
image_paths = []
labels = []
for index, row in labels_df.iterrows():
    image_filename = row['id_code']
    label = row['diagnosis']
    image_path = os.path.join(base_image_dir, f"{label_map[label]}/{image_filename}.png")
    image_paths.append(image_path)
    labels.append(label)

In [4]:
# Split the dataset into training and validation sets
X_train_paths, X_val_paths, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Define the image size and batch size
image_size = (224, 224)
batch_size = 32

In [5]:
# Function to preprocess a single image
def preprocess_image(image_path, target_size):
    """Load and preprocess a single image."""
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize to [0, 1]
    return image

In [6]:
# Data generator for training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def train_generator(X_paths, y, batch_size, image_size):
    while True:
        for start in range(0, len(X_paths), batch_size):
            end = min(start + batch_size, len(X_paths))
            X_batch_paths = X_paths[start:end]
            y_batch = y[start:end]
            X_batch = np.array([preprocess_image(img_path, image_size) for img_path in X_batch_paths])
            yield X_batch, np.array(y_batch)

In [7]:
# Data generator for validation
val_datagen = ImageDataGenerator()

def val_generator(X_paths, y, batch_size, image_size):
    while True:
        for start in range(0, len(X_paths), batch_size):
            end = min(start + batch_size, len(X_paths))
            X_batch_paths = X_paths[start:end]
            y_batch = y[start:end]
            X_batch = np.array([preprocess_image(img_path, image_size) for img_path in X_batch_paths])
            yield X_batch, np.array(y_batch)

In [8]:
# Load the ResNet50 model without the top layer
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Freeze the layers in the base model
for layer in resnet_base.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50
x = GlobalAveragePooling2D()(resnet_base.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(5, activation='softmax')(x)  # Assuming 5 classes for the diabetic retinopathy levels

# Create the model
model = Model(inputs=resnet_base.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [9]:
# Train the model
history = model.fit(
    train_generator(X_train_paths, y_train, batch_size, image_size),
    steps_per_epoch=len(X_train_paths) // batch_size,
    validation_data=val_generator(X_val_paths, y_val, batch_size, image_size),
    validation_steps=len(X_val_paths) // batch_size,
    epochs=10  # Adjust the number of epochs as needed
)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - accuracy: 0.4910 - loss: 1.3299 - val_accuracy: 0.6193 - val_loss: 0.9931
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 89s 987ms/step - accuracy: 0.6162 - loss: 1.0010 - val_accuracy: 0.6576 - val_loss: 0.9788
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.6377 - loss: 0.9683 - val_accuracy: 0.6648 - val_loss: 0.9695
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.6197 - loss: 0.9815 - val_accuracy: 0.6805 - val_loss: 0.9418
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.6428 - loss: 0.9833 - val_accuracy: 0.6790 - val_loss: 0.9219
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 90s 985ms/step - accuracy: 0.6394 - loss: 0.9649 - val_accuracy: 0.6776 - val_loss: 0.9247
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 86s 948ms/step - accuracy: 0.6465 - loss: 0.9493 - val_accuracy: 0.6762 - val_loss: 0.9226
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 84s 929ms/step - accuracy: 0.6466 - loss: 0.9555 - val_accuracy: 

In [10]:
# Save the model
model.save('../models/resnet50_model.keras')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator(X_val_paths, y_val, batch_size, image_size), steps=len(X_val_paths) // batch_size)
print(f"Validation Accuracy: {val_accuracy}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 703ms/step - accuracy: 0.6981 - loss: 0.8885
Validation Accuracy: 0.6789772510528564
